# Cylindrical Magneto-Coriolis Modes

This notebook builds a complete symbolic-to-numerical pipeline to compute linear **magneto-Coriolis modes** in a rotating cylinder using a **toroidal–poloidal** formulation. Velocity and magnetic perturbations are expanded as Fourier modes $e^{i(k z + m\varphi - \omega t)}$ with radial amplitudes defined by the potentials $(T,P)$ for the flow and $(G,F)$ for the magnetic field. With SageMath’s differential operators in cylindrical coordinates, we construct the linearized momentum and induction equations, and assemble a coupled radial system for $[T,P,G,F]^T$. Boundary conditions at the cylinder radius $s=a$ are imposed by matching the interior field to an exterior current-free field $\boldsymbol B_{\text{ext}}=-\nabla\Phi$. The resulting problem is a generalized eigenvalue problem, which is solved numerically (SciPy) to obtain radial eigenvalues and associated spectra $\omega\,(m, n, k)$.

## 1. Definitions

In [1]:
version()

'SageMath version 10.2, Release Date: 2023-12-03'

In [2]:
%display latex 

In [3]:
E.<s,ph,z> = EuclideanSpace(coordinates='cylindrical')
print(E)
E

Euclidean space E^3


Euclidean space E^3

In [4]:
var('k');
var('m');
var('ω');
var('Omega');
var('t');
var('B0');
var('a');
var('mu');
var('rho');
var('C1');

In [8]:
e_s=E.cylindrical_frame()[1]
e_ph=E.cylindrical_frame()[2]
e_z=E.cylindrical_frame()[3]

### Toroidal - Poloidal Fields

In [9]:
T = E.scalar_field(function('T')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='T')
T.display()

T: E^3 → ℝ
   (s, ph, z) ↦ T(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [10]:
G = E.scalar_field(function('G')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='G')
G.display()

G: E^3 → ℝ
   (s, ph, z) ↦ G(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [11]:
P = E.scalar_field(function('P')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='P')
P.display()

P: E^3 → ℝ
   (s, ph, z) ↦ P(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [12]:
F = E.scalar_field(function('F')(s)*exp(I*k*z)*exp(I*m*ph)*exp(I*ω*t), name='F')
F.display()

F: E^3 → ℝ
   (s, ph, z) ↦ F(s)*e^(I*m*ph + I*k*z + I*t*ω)

In [13]:
from sage.manifolds.operators import *
from sage.manifolds.operators import grad

## 2. Hydrodynamic

### Velocity field

In [14]:
v= curl(T*e_s) + curl(curl(P*e_s))
v.display()

curl(T*e_s)+curl(curl(P*e_s)) = (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*P(s)*e^(I*k*z + I*t*ω)/s^2 e_s + (I*k*s^2*T(s)*e^(I*m*ph) + I*m*s*e^(I*m*ph)*d(P)/ds - I*m*P(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s^2 e_ph + (I*k*s*e^(I*m*ph)*d(P)/ds + I*k*P(s)*e^(I*m*ph) - I*m*T(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s e_z

 \begin{align}
    v_s &= \left( k^2 P(s) + \frac{m^2}{s^2}P(s) \right) e^{i(kz + m\varphi + \omega t)}  \\
    v_{\varphi} &= \left(ik T(s) + \frac{im}{s}\frac{\partial P}{\partial s} - \frac{im}{s^2}P(s) \right) e^{i(kz + m\varphi + \omega t)} \\
    v_z &= \left( ik \frac{\partial P}{\partial s} + \frac{ik}{s}P(s) - \frac{im}{s}T(s) \right) e^{i(kz + m\varphi + \omega t)}
\end{align}

In [15]:
dt_v = I*ω*v
dt_v.display()

(I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*P(s)*e^(I*k*z + I*t*ω)/s^2 e_s - (k*s^2*T(s)*e^(I*m*ph) + m*s*e^(I*m*ph)*d(P)/ds - m*P(s)*e^(I*m*ph))*ω*e^(I*k*z + I*t*ω)/s^2 e_ph - (k*s*e^(I*m*ph)*d(P)/ds + k*P(s)*e^(I*m*ph) - m*T(s)*e^(I*m*ph))*ω*e^(I*k*z + I*t*ω)/s e_z

### Navier-Stokes equations

In [12]:
NS_vec = dt_v + 2*Omega*e_z.cross(v)
NS_vec.display()

((I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*P(s)*e^(I*k*z) - 2*(I*Omega*k*s^2*T(s)*e^(I*m*ph) + I*Omega*m*s*e^(I*m*ph)*d(P)/ds - I*Omega*m*P(s)*e^(I*m*ph))*e^(I*k*z))*e^(I*t*ω)/s^2 e_s - ((k*s^2*T(s)*e^(I*m*ph) + m*s*e^(I*m*ph)*d(P)/ds - m*P(s)*e^(I*m*ph))*ω*e^(I*k*z) - 2*(Omega*k^2*s^2*e^(I*m*ph) + Omega*m^2*e^(I*m*ph))*P(s)*e^(I*k*z))*e^(I*t*ω)/s^2 e_ph - (k*s*e^(I*m*ph)*d(P)/ds + k*P(s)*e^(I*m*ph) - m*T(s)*e^(I*m*ph))*ω*e^(I*k*z + I*t*ω)/s e_z

In [13]:
NS_1 = e_s.dot(curl(NS_vec))
NS_1.display()

E^3 → ℝ
(s, ph, z) ↦ -((2*I*k*m*P(s)*e^(I*m*ph) + (-I*k^2*s^2*e^(I*m*ph) - I*m^2*e^(I*m*ph))*T(s))*ω*e^(I*k*z) + 2*(I*Omega*k^3*s^2*e^(I*m*ph) + I*Omega*k*m^2*e^(I*m*ph))*P(s)*e^(I*k*z))*e^(I*t*ω)/s^2

In [14]:
NS_2 = e_s.dot(curl(curl(NS_vec)))
NS_2.display()

E^3 → ℝ
(s, ph, z) ↦ ((-2*I*k*m*s^2*T(s)*e^(I*m*ph) + (I*k^4*s^4*e^(I*m*ph) + (2*I*k^2*m^2 + I*k^2)*s^2*e^(I*m*ph) + (I*m^4 - I*m^2)*e^(I*m*ph))*P(s) + (-I*k^2*s^3*e^(I*m*ph) + I*m^2*s*e^(I*m*ph))*d(P)/ds + (-I*k^2*s^4*e^(I*m*ph) - I*m^2*s^2*e^(I*m*ph))*d^2(P)/ds^2)*ω*e^(I*k*z) + 2*(2*I*Omega*k^2*m*s^2*P(s)*e^(I*m*ph) + (-I*Omega*k^3*s^4*e^(I*m*ph) - I*Omega*k*m^2*s^2*e^(I*m*ph))*T(s))*e^(I*k*z))*e^(I*t*ω)/s^4

### Eigenvalue system

In [15]:
NS_1_eq = (NS_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
NS_1_eq.full_simplify().show()

(-2*I*k*m*P(s) + (I*k^2*s^2 + I*m^2)*T(s))*ω + 2*(-I*Omega*k^3*s^2 - I*Omega*k*m^2)*P(s) == 0

In [16]:
NS_2_eq = (NS_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
NS_2_eq.full_simplify().show()

(4*I*Omega*k^2*m*s^2*P(s) + (-2*I*k*m*s^2*T(s) - (-I*k^4*s^4 - I*m^4 + (-2*I*k^2*m^2 - I*k^2)*s^2 + I*m^2)*P(s) - (I*k^2*s^3 - I*m^2*s)*diff(P(s), s) - (I*k^2*s^4 + I*m^2*s^2)*diff(P(s), s, s))*ω - 2*(I*Omega*k^3*s^4 + I*Omega*k*m^2*s^2)*T(s))/s^2 == 0

\begin{equation}
A(s) \begin{bmatrix}
T(s) \\
P(s) \end{bmatrix} 
= \omega B(s)
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
\end{equation}


\begin{equation}
\begin{bmatrix}
0 & -2(i \Omega k^3 s^2 + i \Omega k m^2) \\
-2(i \Omega k^{3} s^{2} + i \Omega k m^{2}) & -4i \Omega k^2 m \\
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
= \omega 
\begin{bmatrix}
i(k^2 s^2 + m^2) & 2 i k m \\
2 i k m &
\dfrac{-(-i k^4 s^4 - i m^4 + (-2 i k^2 m^2 - i k^2) s^2 + i m^2) - (i k^2 s^3 - i m^2 s) \dfrac{\partial}{\partial s} - (i k^2 s^4 + i m^2 s^2) \dfrac{\partial ^2}{\partial s^2}}{s^2}
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \end{bmatrix}
\end{equation}


## 3. Greenspan's solution : $B_0 = 0$

We need to find the solutions $\xi$ of the equation:
\begin{equation}
    \xi\frac{d}{d\xi}J_{\lvert m \rvert}(\xi) + m\left( 1 + \frac{\xi^2 h^2}{n_{ax}^2\pi^2 a^2}\right)^{1/2} J_{\lvert m \rvert}(\xi) = 0
\end{equation}
With the corresponding eigenvalues : 
\begin{equation}
    \lambda_{knm} = 2 \left( 1 + \frac{\xi^2_{knm}h^2}{n_{ax}^2\pi^2 a^2}\right)^{-1/2}
\end{equation}

## 4. Magnetohydrodynamic

### Magnetic field

In [19]:
b = curl(G*e_s) + curl(curl(F*e_s))
b.display()

curl(G*e_s)+curl(curl(F*e_s)) = (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*F(s)*e^(I*k*z + I*t*ω)/s^2 e_s + (I*k*s^2*G(s)*e^(I*m*ph) + I*m*s*e^(I*m*ph)*d(F)/ds - I*m*F(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s^2 e_ph + (I*k*s*e^(I*m*ph)*d(F)/ds + I*k*F(s)*e^(I*m*ph) - I*m*G(s)*e^(I*m*ph))*e^(I*k*z + I*t*ω)/s e_z

In [20]:
b_s = e_s.dot(b)
b_s.display()

e_s.curl(G*e_s)+curl(curl(F*e_s)): E^3 → ℝ
   (s, ph, z) ↦ (k^2*s^2*e^(I*m*ph) + m^2*e^(I*m*ph))*F(s)*e^(I*k*z + I*t*ω)/s^2

In [20]:
dt_b = I*ω*b

In [21]:
dt_b_s = e_s.dot(dt_b)
dt_b_s.display()

E^3 → ℝ
(s, ph, z) ↦ (I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*F(s)*e^(I*k*z + I*t*ω)/s^2

### Equations of motion

#### Magnetic tension

We need to calculate $\nabla \times (ikB_0 \vec{b})\lvert_s$ and $\nabla \times (\nabla \times ikB_0 \vec{b})\lvert_s$ to add to the N-S equations.

In [34]:
MHD_1 = NS_1 - (1/mu*rho)*e_s.dot(curl(I*k*B0*b))

In [37]:
MHD_1_eq = (MHD_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
MHD_1_eq.full_simplify().show()

(2*I*B0*k^2*m*rho*F(s) + (-2*I*k*m*mu*P(s) + (I*k^2*mu*s^2 + I*m^2*mu)*T(s))*ω + (-I*B0*k^3*rho*s^2 - I*B0*k*m^2*rho)*G(s) + 2*(-I*Omega*k^3*mu*s^2 - I*Omega*k*m^2*mu)*P(s))/mu == 0

In [35]:
MHD_2 = NS_2 - (1/mu*rho)*e_s.dot(curl(curl(I*k*B0*b)))

In [38]:
MHD_2_eq = (MHD_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
MHD_2_eq.full_simplify().show()

(2*I*B0*k^2*m*rho*s^2*G(s) + 4*I*Omega*k^2*m*mu*s^2*P(s) + (-2*I*k*m*mu*s^2*T(s) - (-I*k^4*mu*s^4 + (-2*I*k^2*m^2 - I*k^2)*mu*s^2 + (-I*m^4 + I*m^2)*mu)*P(s) - (I*k^2*mu*s^3 - I*m^2*mu*s)*diff(P(s), s) - (I*k^2*mu*s^4 + I*m^2*mu*s^2)*diff(P(s), s, s))*ω - (I*B0*k^5*rho*s^4 + (2*I*B0*k^3*m^2 + I*B0*k^3)*rho*s^2 + (I*B0*k*m^4 - I*B0*k*m^2)*rho)*F(s) - 2*(I*Omega*k^3*mu*s^4 + I*Omega*k*m^2*mu*s^2)*T(s) - (-I*B0*k^3*rho*s^3 + I*B0*k*m^2*rho*s)*diff(F(s), s) - (-I*B0*k^3*rho*s^4 - I*B0*k*m^2*rho*s^2)*diff(F(s), s, s))/(mu*s^2) == 0

### Ideal induction

\begin{equation}
    \frac{\partial}{\partial t}\vec{b}  = \left( \vec{B}_0 \cdot \nabla\right)\vec{v}
\end{equation}

In [23]:
Ind_1 = e_s.dot(dt_b - B0*I*k*v)
Ind_1.display()

E^3 → ℝ
(s, ph, z) ↦ ((I*k^2*s^2*e^(I*m*ph) + I*m^2*e^(I*m*ph))*ω*F(s)*e^(I*k*z) + (-I*B0*k^3*s^2*e^(I*m*ph) - I*B0*k*m^2*e^(I*m*ph))*P(s)*e^(I*k*z))*e^(I*t*ω)/s^2

In [29]:
Ind_1_eq = (Ind_1.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
Ind_1_eq.full_simplify().show()

(I*k^2*s^2 + I*m^2)*ω*F(s) + (-I*B0*k^3*s^2 - I*B0*k*m^2)*P(s) == 0

In [24]:
Ind_2 = e_s.dot(curl(dt_b - B0*I*k*v))
Ind_2.display()

E^3 → ℝ
(s, ph, z) ↦ -((2*I*k*m*F(s)*e^(I*m*ph) + (-I*k^2*s^2*e^(I*m*ph) - I*m^2*e^(I*m*ph))*G(s))*ω*e^(I*k*z) + (-2*I*B0*k^2*m*P(s)*e^(I*m*ph) + (I*B0*k^3*s^2*e^(I*m*ph) + I*B0*k*m^2*e^(I*m*ph))*T(s))*e^(I*k*z))*e^(I*t*ω)/s^2

In [31]:
Ind_2_eq = (Ind_2.expr() == 0)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
Ind_2_eq.full_simplify().show()

2*I*B0*k^2*m*P(s) + (-2*I*k*m*F(s) + (I*k^2*s^2 + I*m^2)*G(s))*ω + (-I*B0*k^3*s^2 - I*B0*k*m^2)*T(s) == 0

### Eigenvalue system

\begin{equation}
\mathcal{A(s)} \begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix} 
= \omega \mathcal{B(s)}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
\end{equation}


\begin{equation}
\begin{bmatrix}
0 & -2 i \Omega k (k^2 s^2 + m^2) & - i B_0 \rho k (k^2 s^2 + m^2) & + 2 i B_0 k^2 m \rho \\
0 & 4 i \Omega k^2 m & \frac{1}{\mu s^2} 2 i B_0 k^2 m \rho s^2 & \frac{-1}{\mu s^2} \left(i B_0 k^5 \rho s^4 + (2 i B_0 k^3 m^2 + i B_0 k^3) \rho s^2 + (i B_0 k m^4 - i B_0 k m^2) \rho\right) + \frac{-1}{\mu s^2} \left(-i B_0 k^3 \rho s^3 + i B_0 k m^2 \rho s\right) \frac{d}{ds} + \frac{-1}{\mu s^2} \left(-i B_0 k^3 \rho s^4 - i B_0 k m^2 \rho s^2\right) \frac{d^2}{ds^2} \\
0 & -i B_0 k (k^2 s^2 + m^2) & 0 & 0 \\
-i B_0 k (k^2 s^2 + m^2) & 2 i B_0 k^2 m & 0 & 0
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
= \omega 
\begin{bmatrix}
i (k^2 s^2 + m^2) & -2 i k m & 0 & 0\\
-2 i k m & i \left(k^4 s^2 + (2k^2 m^2 + k^2) + m^4 - m^2\right) -(i k^2 s + i m^2 1/s) \frac{d}{ds} - (i k^2 s^2 + i m^2) \frac{d^2}{ds^2} & 0 & 0 \\
0 & 0 & 0 & i (k^2 s^2 + m^2) \\
0 & 0 & i (k^2 s^2 + m^2) & -2 i k m
\end{bmatrix}
\begin{bmatrix}
T(s) \\
P(s) \\
G(s) \\
F(s)
\end{bmatrix}
\end{equation}

### Boundary conditions :

A magnetic flux tube is surrounded by an external magnetic field $\mathbf{B}_{ext}$, which can be assumed to be derived from a potential of the same form as toroidal-poloidal fields. That is, we have 
\begin{equation}
    \mathbf{B}_{ext} = - \nabla \Phi \,\ \text{where} \,\ \Phi = \tilde{\Phi}(s) e^{i(m\varphi + kz + \omega t)}
\end{equation}
Where $\tilde{\Phi}$ is again the Fourier amplitude. Since $\nabla \cdot \mathbf{B} = 0$, we can solve
\begin{equation}
    \nabla^2 \Phi = 0
\end{equation}
such that 
\begin{equation}
    \Phi = C_1 Y_m (-iks) e^{i(m \varphi + kz + \omega t)}
\end{equation}

In [24]:
bs = e_s.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bs.expr().full_simplify().show()

(k^2*s^2 + m^2)*F(s)

In [50]:
bph = e_ph.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bph.expr().full_simplify().show()

I*k*s^2*G(s) + I*m*s*diff(F(s), s) - I*m*F(s)

In [26]:
bz = e_z.dot(b)*exp(-I*k*z)*exp(-I*m*ph)*exp(-I*ω*t)*s**2
bz.expr().full_simplify().show()

I*k*s^2*diff(F(s), s) + I*k*s*F(s) - I*m*s*G(s)

In [1]:
F_1, dF_1, G_1, k, m, C = var('F_1 dF_1 G_1 k m C')

In [2]:
Phi = function('Phi')(s)
Phi == C1*bessel_Y(m,-I*k*s)

Phi(s) == C1*bessel_Y(m, -I*k*s)

In [73]:
eq_s = bs + C1*Phi.diff(s)
eq1 = (eq_s.expr() == 0)
eq1.show()

(k^2*s^2 + m^2)*F(s) + C1*diff(Phi(s), s) == 0

In [17]:
F_1 == -(C/(k**2 + m**2))*diff(bessel_Y(m,-(I*k)*s),s)

F_1 == -1/2*I*C*k*(bessel_Y(m + 1, -I*k*s) - bessel_Y(m - 1, -I*k*s))/(k^2 + m^2)

In [18]:
dF_1 == -(C/(k**2 + m**2))*diff(bessel_Y(m,-(I*k)*s),s) - C*(bessel_Y(m,-(I*k)*s))

dF_1 == -1/2*I*C*k*(bessel_Y(m + 1, -I*k*s) - bessel_Y(m - 1, -I*k*s))/(k^2 + m^2) - C*bessel_Y(m, -I*k*s)

In [74]:
eq_ph = bph + I*m*C1*Phi.diff(s)
eq2 = (eq_ph.expr() == 0)
eq2.show()

I*k*s^2*G(s) + I*m*s*diff(F(s), s) + I*C1*m*diff(Phi(s), s) - I*m*F(s) == 0

In [75]:
eq_z = bz + I*m*C1*Phi.diff(s)
eq3 = (eq_z.expr() == 0)
eq3.show()

I*k*s^2*diff(F(s), s) + I*k*s*F(s) - I*m*s*G(s) + I*C1*m*diff(Phi(s), s) == 0

## 5. Lundquist's solution : m = 0, $\Omega = 0$

For $m=0$ and $\Omega \to 0$, the governing equations reduce to Lundquist’s ideal-MHD model for a cylindrical plasma column. In this model, the radial equation for the perturbation \(b(s)\) reads
\begin{equation}
\frac{\partial^2 b}{\partial s^2} + \frac{1}{s} \frac{\partial b}{\partial s} + \left(k_L^2 - \frac{1}{s^2}\right) b = 0,
\end{equation}
whose regular solution at \(s=0\) is
\begin{equation}
b(s,t,z) = A\, J_1(k_L s) \, e^{i\omega t + i\alpha z}.
\end{equation}
the boudnary codnitions $b(a) = 0$ leads to :
\begin{equation}
    J_1(k_L a) = 0 \Rightarrow k_L = \frac{j_{1,n}}{a}
\end{equation}
so that
\begin{equation}
    \omega_{Lund}^{(\pm)} = \pm\, v_A\, k_L
\end{equation}